In [85]:
import bs4 as bs
import urllib.request
import re
import nltk
import heapq
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd

In [86]:
def word_freq_count(text):
    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}
    for word in nltk.word_tokenize(text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1
    
    return word_frequencies

def weighted_freq(word_frequencies):
    maximum_frequncy = sum(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    
    return word_frequencies

def sentence_scores_calc(word_frequencies,tokenized_sentence):
    sentence_scores = {}
    for sent in tokenized_sentence:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
    
    return sentence_scores

def summarizer(text):
    word_frequencies = word_freq_count(text)
    word_frequencies = weighted_freq(word_frequencies)
    sentence_list = nltk.sent_tokenize(text)
    sentence_scores = sentence_scores_calc(word_frequencies, sentence_list)
    summary_sentences = heapq.nlargest(12, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)
    return summary

In [102]:
df = pd.read_excel('C:/Users/praneeth/Downloads/gtd1993_0221dist.xlsx')
corpus = df['summary']

def preprocess_function(raw_text):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(raw_text)
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    porter = PorterStemmer()
    stems = []
    for t in tokens:
        stems.append(porter.stem(t))
    return stems

corpus_array = []
for i in range(0,len(df)):
    word = preprocess_function(str(df['summary'][i]))
    corpus_array.append(word)

final_corpus = []
for i in range(0,len(corpus_array)):
    for ele in corpus_array[i]:
        if ele not in final_corpus:
            final_corpus.append(ele)
ref_corpus = [item for item in final_corpus if not item.isdigit()]

In [103]:
def getTextFromURL(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return text

In [115]:
raw_text = getTextFromURL('https://www.thequint.com/news/world/unsc-skips-taliban-reference-in-statement-on-terror')
summary = summarizer(raw_text)
preprocessed_text = preprocess_function(summary)
score = 0
for i in preprocessed_text:
    if i in ref_corpus:
        score = score + 1

final_score = (score/len(preprocessed_text)) * 100
final_score

36.206896551724135

In [125]:
from googlesearch import search
query = "Taliban"
for j in search(query, tld="co.in", num=10, stop=10, pause=2):
    raw_text = getTextFromURL(j)
    summary = summarizer(raw_text)
    preprocessed_text = preprocess_function(summary)
    score = 0
    for i in preprocessed_text:
        if i in ref_corpus:
            score = score + 1
    if len(preprocessed_text) == 0:
        final_score = 0
    else:
        final_score = (score/len(preprocessed_text)) * 100
    print(j, end = ' ')
    print("Terror Score: ", end = ' ')
    print(final_score)

https://www.moneycontrol.com/news/world/afghanistan-taliban-crisis-live-updates-joe-biden-attends-grim-homecoming-for-us-troops-killed-in-afghanistan-attack-7404891.html Terror Score:  61.04651162790697
https://www.ndtv.com/world-news/taliban-supreme-leader-hibatullah-akhundzada-in-afghanistans-kandahar-city-says-spokesman-report-2523108 Terror Score:  46.794871794871796
https://www.business-standard.com/article/current-affairs/afghanistan-crisis-live-updates-terror-attack-taliban-farmers-india-top-highlights-121083000021_1.html Terror Score:  36.666666666666664
https://en.wikipedia.org/wiki/Taliban Terror Score:  40.816326530612244
https://en.wikipedia.org/wiki/Taliban_insurgency Terror Score:  54.54545454545454
https://en.wikipedia.org/wiki/History_of_Taliban Terror Score:  51.515151515151516
https://en.wikipedia.org/wiki/Tehrik-i-Taliban_Pakistan Terror Score:  50.77720207253886
https://en.wikipedia.org/wiki/Deobandi Terror Score:  42.38095238095238
https://www.bbc.com/news/world-so

In [112]:
score = 0
for i in preprocessed_text:
    if i in ref_corpus:
        score = score + 1

print(score)

21


In [113]:
len(preprocessed_text)

58

In [114]:
score / len(preprocessed_text)

0.3620689655172414